In [ ]:
# 모델 정의 (ResNet18 사전학습 3채널 그대로 사용)
class CanSimModel(nn.Module):
    def __init__(self, tabular_dim=3, hidden_dim=64, num_drugs=4):
        super().__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        # self.backbone.conv1 = ...  (삭제: 1채널로 바꾸지 않음)
        self.backbone.fc = nn.Identity()

        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, hidden_dim),
            nn.ReLU()
        )
        self.fusion = nn.Sequential(
            nn.Linear(512 + hidden_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_drugs)
        )

    def forward(self, img, tabular):
        img_feat = self.backbone(img)
        tab_feat = self.tabular(tabular)
        concat = torch.cat([img_feat, tab_feat], dim=1)
        return self.fusion(concat)  # logits